In [1]:
!pip install BeautifulSoup4

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import string
import re
# url = 'https://yandex.com.am/weather/ru-RU/details?lat=56.48464&lon=84.947649&lang=ru&via=ms'
# url = 'https://yandex.com.am/weather/?lat=56.48464&lon=84.947649'
# url = 'https://yandex.com.am/weather/?lat=55.75581741&lon=37.61764526'
# url = 'https://meteoinfo.ru/forecasts/russia/tomsk-area/tomsk'
area = input('Введите область: ')
city = input('Введите город: ')
url = f'https://meteoinfo.ru/forecasts/russia/{area}-area/{city}'
print(url)

Введите область: Tomsk
Введите город: Tomsk
https://meteoinfo.ru/forecasts/russia/Tomsk-area/Tomsk


In [3]:
response = requests.get(url)
if response.status_code == 200:
  print(f'Данные получены, ответ {response.status_code}. Начинаю парсить!')
  soup = BeautifulSoup(response.text, 'html.parser')
  # soup = BeautifulSoup(response.text, 'lxml')
else:
  print(f'Что-то пошло не так, ответ {response.status_code}.')

Данные получены, ответ 200. Начинаю парсить!


In [4]:
table = soup.find('table', {'class': 'fc_tab_1'})
rows = table.findAll('tr')

In [5]:
# Даты
dates = []
# Дни недели
weekdays = []
# Дни и ночи
t_days = rows[0].findAll('td')
for day in t_days:
  if day.text:
    index = (day.text.index([num for num in day.text if num in string.digits][0]))
    if dates:
      weekdays.append(day.text[:index])
      weekdays.append(day.text[:index])
      dates.append(day.text[index:])
      dates.append(day.text[index:])
    else:
      weekdays.append(day.text[:index])
      dates.append(day.text[index:])
print(dates)
print(weekdays)

['25 августа', '26 августа', '26 августа', '27 августа', '27 августа', '28 августа', '28 августа', '29 августа', '29 августа', '30 августа', '30 августа', '31 августа', '31 августа']
['Воскресенье', 'Понедельник', 'Понедельник', 'Вторник', 'Вторник', 'Среда', 'Среда', 'Четверг', 'Четверг', 'Пятница', 'Пятница', 'Суббота', 'Суббота']


In [6]:
# Облачность, осадки (день)
clouds = []
t_clouds = rows[1].findAll('td')
# Дни
days_nights = [t_clouds[0].text] * 7
for cloud in t_clouds:
  match = re.findall(r'(title=".+")', str(cloud))
  if match:
    clouds.append(match[0].replace('title=', '').replace('"', ''))
print(clouds)
print(days_nights)

['Облачно, дождь', 'Облачно, дождь', 'Облачно, небольшой дождь', 'Облачно, дождь', 'Переменная облачность, небольшой кратковременный дождь', 'Облачно, дождь', 'Облачно, дождь']
['День', 'День', 'День', 'День', 'День', 'День', 'День']


In [7]:
# Температура (день)
temps = []
t_temp = rows[2].findAll('td')
for temp_max in t_temp:
  if 'T максимальная' not in temp_max.text and temp_max.text != '':
    temps.append(temp_max.text.replace('°', ''))
print(temps)

['23', '19', '19', '19', '22', '19', '17']


In [8]:
# Вероятность осадков (день)
precs = []
t_precs = rows[4].findAll('td')
for prec in t_precs:
  if 'Осадки, вероятность' not in prec.text and prec.text != '':
    precs.append(prec.text.replace('%', ''))
print(precs)

['85', '95', '84', '87', '83', '79', '75']


In [9]:
# Направление ветра (день)
wind_directions = []
# Скорость ветра (день)
wind_speeds = []
t_winds = rows[5].findAll('td')
for wind in t_winds:
  match = re.findall(r'(title=".{1,3}")', str(wind))
  if match:
    wind_directions.append(match[0].replace('title=', '').replace('"', ''))
    wind_speeds.append(wind.text.replace(' ', ''))
print(wind_directions)
print(wind_speeds)

['Ю', 'Ю-З', 'Ю-З', 'З', 'Ю-З', 'Ю-З', 'С-З']
['3', '4', '3', '2', '3', '4', '2']


In [10]:
# Атмосферное давление (день)
presss = []
t_press = rows[6].findAll('td')
for press in t_press:
  if 'Давление, мм рт.ст' not in press.text and press.text != '':
    presss.append(press.text)
print(presss)

['740', '742', '746', '748', '746', '744', '746']


In [11]:
t_clouds = rows[7].findAll('td')
# Ночи
for index in range(1, 13, 2):
  days_nights.insert(index, t_clouds[0].text)
print(days_nights)

# Облачность, осадки (ночь)
index = 1
for cloud in t_clouds:
  match = re.findall(r'(title=".+")', str(cloud))
  if match:
    clouds.insert(index, match[0].replace('title=', '').replace('"', ''))
    index += 2
print(clouds)

['День', 'Ночь', 'День', 'Ночь', 'День', 'Ночь', 'День', 'Ночь', 'День', 'Ночь', 'День', 'Ночь', 'День']
['Облачно, дождь', 'Облачно, дождь', 'Облачно, дождь', 'Облачно, дождь', 'Облачно, небольшой дождь', 'Облачно, небольшой дождь', 'Облачно, дождь', 'Переменная облачность, небольшой кратковременный дождь', 'Переменная облачность, небольшой кратковременный дождь', 'Облачно, временами небольшой дождь', 'Облачно, дождь', 'Облачно, кратковременный дождь', 'Облачно, дождь']


In [12]:
# Температура (ночь)
t_temp = rows[8].findAll('td')
index = 1
for temp_max in t_temp:
  if 'T минимальная' not in temp_max.text and temp_max.text != '':
    temps.insert(index, temp_max.text.replace('°', ''))
    index += 2
print(temps)

['23', '13', '19', '13', '19', '13', '19', '11', '22', '14', '19', '11', '17']


In [13]:
# Вероятность осадков (ночь)
t_precs = rows[10].findAll('td')
index = 1
for prec in t_precs:
  if 'Осадки, вероятность' not in prec.text and prec.text != '':
    precs.insert(index, prec.text.replace('%', ''))
    index += 2
print(precs)

['85', '95', '95', '91', '84', '73', '87', '83', '83', '79', '79', '75', '75']


In [14]:
# Направление ветра (ночь)
# Скорость ветра (ночь)
t_winds = rows[11].findAll('td')
index = 1
for wind in t_winds:
  match = re.findall(r'(title=".{1,3}")', str(wind))
  if match:
    wind_directions.insert(index, match[0].replace('title=', '').replace('"', ''))
    wind_speeds.insert(index, wind.text.replace(' ', ''))
    index += 2
print(wind_directions)
print(wind_speeds)

['Ю', 'Ю-З', 'Ю-З', 'Ю-З', 'Ю-З', 'Ю-З', 'З', 'Ю-З', 'Ю-З', 'Ю', 'Ю-З', 'З', 'С-З']
['3', '4', '4', '3', '3', '2', '2', '2', '3', '4', '4', '1', '2']


In [15]:
# Атмосферное давление (ночь)
t_press = rows[12].findAll('td')
index = 1
for press in t_press:
  if 'Давление, мм рт.ст' not in press.text and press.text != '':
    presss.insert(index, press.text)
    index += 2
print(presss)

['740', '739', '742', '744', '746', '747', '748', '748', '746', '745', '744', '745', '746']


In [16]:
weather = {'Area': [area] * len(dates), 'City': [city] * len(dates), 'Date': dates, 'Weekday': weekdays, 'Day/Night': days_nights, 'Cloudiness': clouds, 'Temperature': temps, 'PrecipitationProbability': precs, 'WindDirection': wind_directions, 'WindSpeed': wind_speeds, 'AtmosphericPressure': presss}
df = pd.DataFrame(weather)

In [17]:
df

,Area,City,Date,Weekday,Day/Night,Cloudiness,Temperature,PrecipitationProbability,WindDirection,WindSpeed,AtmosphericPressure
0,Tomsk,Tomsk,25 августа,Воскресенье,День,"Облачно, дождь",23,85,Ю,3,740
1,Tomsk,Tomsk,26 августа,Понедельник,Ночь,"Облачно, дождь",13,95,Ю-З,4,739
2,Tomsk,Tomsk,26 августа,Понедельник,День,"Облачно, дождь",19,95,Ю-З,4,742
3,Tomsk,Tomsk,27 августа,Вторник,Ночь,"Облачно, дождь",13,91,Ю-З,3,744
4,Tomsk,Tomsk,27 августа,Вторник,День,"Облачно, небольшой дождь",19,84,Ю-З,3,746
5,Tomsk,Tomsk,28 августа,Среда,Ночь,"Облачно, небольшой дождь",13,73,Ю-З,2,747
6,Tomsk,Tomsk,28 августа,Среда,День,"Облачно, дождь",19,87,З,2,748
7,Tomsk,Tomsk,29 августа,Четверг,Ночь,"Переменная облачность, небольшой кратковременн...",11,83,Ю-З,2,748
8,Tomsk,Tomsk,29 августа,Четверг,День,"Переменная облачность, небольшой кратковременн...",22,83,Ю-З,3,746
9,Tomsk,Tomsk,30 августа,Пятница,Ночь,"Облачно, временами небольшой дождь",14,79,Ю,4,745
